## Flow routing and watershed analysis
Resources:

* [
GRASS GIS overview and manual](http://grass.osgeo.org/grass72/manuals/index.html)
*  [Recommendations](data_acquisition.html#commands)
and [tutorial](http://www4.ncsu.edu/~akratoc/GRASS_intro/)
how to use GUI from the first assignment



Text file with outlet point coordinates:

* point coordinates ([outlet_point.txt](data/outlet_point.txt))


### Start GRASS GIS
In startup pannel set GIS Data Directory to path to datasets,
for example on MS Windows, `C:\Users\myname\grassdata`.
For Project location select nc_spm_08_grass7 (North Carolina, State Plane, meters) and
for Accessible mapset create a new mapset (called e.g. HW_hydrology) and
click Start GRASS.

In [ ]:
# using Python to initialize GRASS GIS
import os
import sys
import subprocess
from IPython.display import Image

# create GRASS GIS runtime environment
gisbase = subprocess.check_output(["grass", "--config", "path"]).strip()
os.environ['GISBASE'] = gisbase
sys.path.append(os.path.join(gisbase, "etc", "python"))

# do GRASS GIS imports
import grass.script as gs
import grass.script.setup as gsetup

# set GRASS GIS session data
rcfile = gsetup.init(gisbase, "/home/jovyan/grassdata", "nc_spm_08_grass7", "user1")

In [ ]:
# using Python to initialize GRASS GIS
# default font displays
os.environ['GRASS_FONT'] = 'sans'
# overwrite existing maps
os.environ['GRASS_OVERWRITE'] = '1'
gs.set_raise_on_error(True)
gs.set_capture_stderr(True)

In [ ]:
# using Python to initialize GRASS GIS
# set display modules to render into a file (named map.png by default)
os.environ['GRASS_RENDER_IMMEDIATE'] = 'cairo'
os.environ['GRASS_RENDER_FILE_READ'] = 'TRUE'
os.environ['GRASS_LEGEND_FILE'] = 'legend.txt'

Change working directory:
_Settings_ > _GRASS working environment_ > _Change working directory_ > select/create any directory
or type `cd` (stands for change directory) into the GUI
_Console_ and hit Enter:

In [ ]:
# a proper directory is already set, download files
import urllib
urllib.urlretrieve("http://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/outlet_point.txt", "outlet_point.txt")

Download all text files with site locations (see above)
to the selected directory. Now you can use the commands from the assignment requiring the text file
without the need to specify the full path to the file.



start GRASS

### Compute flow direction, flow accumulation and subwatersheds
Compute flow direction, flow accumulation and subwatersheds
with approx. size of 10000 cells from 30m NED.

In [ ]:
!g.region raster=elev_ned_30m -p
!r.watershed -s elevation=elev_ned_30m threshold=10000 accumulation=accum_10K drainage=draindir_10K basin=basin_10K

Extract more detailed streams from flow accumulation raster:

In [ ]:
!r.mapcalc "streams_der_30m = if(abs(accum_10K) > 100, 1, null())"

Convert to vector format and display results
along with the official Wake county streams (red):

In [ ]:
!r.to.vect -s basin_10K output=basin_10K type=area
!r.thin streams_der_30m output=streams_der_30m_t
!r.to.vect -s streams_der_30m_t out=streams_der_30m type=line

Generate shaded map and display:
How do the derived streams compare with the official stream map?
How can you modify the mapcalc expression to make stream origins fit better
with the official stream map?

In [ ]:
!d.his hue=basin_10K intensity=elevation_shade brighten=40
!d.vect basin_10K type=boundary
!d.rast lakes
!d.vect streams_der_30m color=blue
!d.vect streams color=red
Image(filename="map.png")

### Create map of DEM depressions
Depression filling is often necessary for certain flow routing algorithms but it can alter the elevation data significantly. 
Find out how extensive the depressions are in our DEM.
Note that _r.watershed_ doesn't need any depression filling thanks to
its underlying algorithm which uses least cost path to get over depressions.

In [ ]:
!g.region raster=elevation -p
!r.fill.dir input=elevation output=elev_fill1 direction=dir1 areas=unres1
!r.fill.dir input=elev_fill1 output=elev_fill2 direction=dir2 areas=unres2
!r.fill.dir input=elev_fill2 output=elev_fill3 direction=dir3 areas=unres3
!r.mapcalc "depr_bin = if((elevation-elev_fill3) < 0., 1, null())"

Remove all previously used layers from the _Layer Manager_ and
display the new results, compare the derived depressions with actual lakes:

In [ ]:
!d.erase
!d.rast elevation
!d.vect roadsmajor
!d.rast depr_bin
!d.vect lakes type=area fill_color=aqua
Image(filename="map.png")

### Derive contributing area for a given outlet
Set region to the high resolution study area and zoom to it:

In [ ]:
!g.region rural_1m -p
!r.watershed -as elevation=elev_lid792_1m threshold=5000 accumulation=accum_5K drainage=draindir_5K basin=basin_5K

Remove previous layers. Display extracted streams over aerial image:

In [ ]:
!d.erase
!d.rast ortho_2001_t792_1m
!d.rast accum_5K values=1000-1000000
Image(filename="map.png")

Identify outlet on the extracted stream.

Create a vector map with the point east=638845.52 north=220085.26 (download the text file)
that has accum_5K=224510.

In [ ]:
!v.in.ascii input=outlet_point.txt output=outletA30 separator=space
!d.vect outletA30 color=yellow
Image(filename="map.png")

Delineate the contributing area associated with this outlet and convert it to vector format:

In [ ]:
!r.water.outlet input=draindir_5K output=basin_A30 coordinates=638845.52,220085.26
!r.to.vect -s basin_A30 output=basin_A30 type=area

Display watershed boundary along with contours.
Compute the watershed area (category 1)

In [ ]:
!d.vect basin_A30 type=boundary color=green width=2
!r.contour elev_lid792_1m output=elev_lid792_cont_1m step=1 minlevel=104
!d.vect elev_lid792_cont_1m color=white
Image(filename="map.png")
!r.report basin_A30 unit=h,a
Image(filename="map.png")

### Assess and mitigate impact of the road on flowrouting
Compare the extracted streams (accum > 800) with official stream data:

In [ ]:
!d.rast ortho_2001_t792_1m
!d.rast accum_5K values=800-1000000
!d.vect streets_wake color=red
!d.vect streams color=green
Image(filename="map.png")

### Carve a pre-defined channel given by the stream data into DEM

In [ ]:
!r.carve raster=elev_lid792_1m vector=streams width=2 depth=0.8 output=elev_lidcarved_1m
!r.colors elev_lidcarved_1m raster=elev_lid792_1m
!d.rast elev_lidcarved_1m
Image(filename="map.png")

Extract streams from the carved DEM and compare with the official streams map.
What is the difference between accum_5K, accumc_5K1m and streams?
Explain the advantage and disadvantage of carving.

In [ ]:
!r.watershed -as elevation=elev_lidcarved_1m threshold=50000 accumulation=accumc_5K1m
!d.rast accumc_5K1m values=1500-10000000
!d.vect streams
Image(filename="map.png")

### Compute refined flow pattern using D-inf
Compare upslope and downslope flow lines:
On what type of landform (ridge, valley) they converge?

Remove or switch off all previous layers.

In [ ]:
!g.region raster=elev_lid792_1m -p
!r.flow elevation=elev_lid792_1m flowline=flowlines flowlength=flowlg_1m flowaccumulation=flowacc_1m
!r.flow -u elevation=elev_lid792_1m flowlength=flowlgup_1m flowaccumulation=flowaccup_1m

Display maps along with contours to see relation to terrain:

In [ ]:
!d.rast flowacc_1m
!d.vect elev_lid792_cont_1m color=red
!d.rast flowaccup_1m
!d.vect elev_lid792_cont_1m color=red
!d.vect flowlines
Image(filename="map.png")

### Compare the multiple flow direction result with D8 and Dinf
Compare the result of accum_5K and accum_mfd_5K1m. Why are they different,
which is more accurate and why?

In [ ]:
!r.watershed -a elevation=elev_lid792_1m threshold=50000 accumulation=accum_mfd_5K1m
!d.rast accum_mfd_5K1m
!d.vect streams
Image(filename="map.png")

### Compute wetness index

In [ ]:
!g.region rural_1m -p
!r.topidx elev_lid792_1m output=wetness_1m
!d.rast wetness_1m
Image(filename="map.png")

Display elev_lid792_1m in 3D and drape over wetness_1m as color.
Note: switch off all layers except for elev_lid792_1m.

### Create a map of flooded area
Create a map of flooded area for a given water level and seed point:

In [ ]:
!r.lake elevation=elev_lid792_1m water_level=113.5 lake=flood1 coordinates=638728,220278
!d.rast elev_lid792_1m
!d.rast flood1
Image(filename="map.png")

Increase water level to 113.7m and 114.0m and create flooded
area maps at these two levels.

In [ ]:
# end the GRASS session
os.remove(rcfile)